Calypso project - option CICE Centrale Nantes
Program of stress calculations for the blades

In [9]:
import numpy as np
import math

Useful Files 

In [11]:
Table = np.load("Section_cord_unitary.npy")
data_blade = np.loadtxt('ChordAngle.dat')  

#tests
Number_points = 100
data_blade = np.empty(Number_points)
distrib = np.linspace(0,50,Number_points)
distrib_x = np.linspace(0,0.8,Number_points)
Cord = 0.12*np.ones(Number_points)
data_blade = np.stack((distrib_x,Cord,distrib))
data_blade=data_blade.T


FileNotFoundError: [Errno 2] No such file or directory: 'Section_cord_unitary.npy'

Useful Values

In [12]:
Area_unitary = Area_unitary_section(Table)
Clalpha = 2*np.pi
pitch = 0
excentration = 0.05
rotation_speed = 83.7
Uvent = 15
rho_material = 1250

NameError: name 'Area_section_unitary' is not defined

Usefull Functions for stress calculations

In [14]:
#Useful functions for the code

def Area_unitary_section (table_interpolation) :
    A = 0
    N = len(table_interpolation[0]) - 1
    for i in range (N):
        A +=  1/N * ((table_interpolation[2][i])-(table_interpolation[1][i]))
    return A

#To calculate the distribution of L

def l_linear (Uvent,pitch,Clalpha,cord,angle):
    return 1/2 * 1.225* (Uvent)**2 * Clalpha * (pitch + angle) * corde*np.pi/180
    
def distrib_l (tab,Uvent,pitch,Clalpha) :
    l = []
    for i in range (len(tab)) :
        l.append([tab[i][0], l_linear(Uvent,pitch,Clalpha,tab[i][1],tab[i][2])])
    return l
    
#To calculate the distribution of f (radial centrifugal force)

def f_linear (rho_material,Area_section,rotation_speed,position_blade,excentration_blade) :
    return rho_material*(position_blade+excentration_blade)*Area_section*(rotation_speed)**2

def distrib_f (tab,rotation_speed,Area_unitary,excentration_blade,rho_material) :
    f = []
    for i in range (len(tab)):
        f.append([tab[i][0],f_linear(rho_material,Area_unitary*tab[i][1]**2,rotation_speed,tab[i][0],excentration_blade)])
    return f

def distrib_m (distrib_l,coordinates_ce,coordinates_ca) :
    m = []
    for i in range(len(distrib_l)) :
        m.append(distrib_l[i]*(coordinates_ca**2+coordinates_ce**2))
    return m

#Useful files



#Useful values


#distributions
L = distrib_l(data_blade,Uvent,pitch,Clalpha)
F = distrib_f(data_blade,rotation_speed,Area_unitary,excentration,rho_material)

#for now
m = []
for i in range (len(L)) :
    m.append([L[i][0],L[i][0]*L[i][1]])


#Integrations

def method_trapeze(tab) :
    integral = 0.0
    for i in range(len(tab) - 1):
        x_current, y_current = tab[i]
        x_next, y_next = tab[i + 1]
        h = x_next - x_current
        integral += (y_current + y_next) * h / 2  # Trapeze formula
    return integral

#Torsion moment
"F_aero is taken from the Fluid table obtained by the fluid department of the project"
def moment_torsion (ce,ca,Faero) : #List of coord. Elastic Center, Aero Center, and liste of aero. stress
    Mtorsion = []
    for i in range (len(ce)) :
        d = np.sqrt ((ce[i][0]-ca[i][0])**2+(ce[i][1]-ca[i][1])**2)
        M_torsion.append(d*Faero[i])
    return Mtorsion

NameError: name 'data_blade' is not defined

Useful functions for geometry calculations

In [15]:
################################
#CALCULATE ON AN UNITARY SECTION
################################

"""Section_interpolated is a three-column table
 1 -> coordinate x
 2 -> coordinate y_lower
 3 -> coordinate y_upper"""

def Calculus_geometrical (Section_interpolated) :
    "Initialisation of Iyy, Izz et Iyz"
    Iyy = 0
    Izz = 0 
    Iyz1 = 0
    
    N = len(Section_interpolated[0]) - 1
    
    "Calculating by summing the integers along all parts of the discretisation"
    for i in range (N) :
        Iyy += 1/(3*N) * ((Section_interpolated[2][i])**3-(Section_interpolated[1][i])**3)
        Izz += 1/N * ((Section_interpolated[2][i])-(Section_interpolated[1][i]))*Section_interpolated[0][i]**2
        Iyz1 += 1/(2*N) * ((Section_interpolated[2][i])**2-(Section_interpolated[1][i])**2)* Section_interpolee[0][i]
    Sy = 0
    Sz = 0
    A = 0
    
    N = len(Section_interpolated[0]) - 1
    
    "Calculating by summing the integers along all parts of the discretisation"
    for i in range (N) :
        Sy += 1/(2*N) * ((Section_interpolated[2][i])**2-(Section_interpolated[1][i])**2)
        Sz += 1/N * ((Section_interpolated[2][i])-(Section_interpolated[1][i]))*Section_interpolated[0][i]
        A +=  1/N * ((Section_interpolated[2][i])-(Section_interpolated[1][i]))
        Zc = Sy/A
        Yc = Sz/A
    Iy = Iyy - (Zc**2)*A
    Iz = Izz - A*Yc**2
    Iyz = Iyz1 - Yc*Zc*A
    I1 = 1/2*(Iz+Iy) - math.sqrt(((Iy-Iz)/2)**2 + Iyz**2)
    I2 = 1/2*(Iz+Iy) + math.sqrt(((Iy-Iz)/2)**2 + Iyz**2)
    Theta = math.atan2(2*Iyz, Iz-Iy)
    print(Theta*180/math.pi)
    
    "returning all useful quantities"
    return Yc, Zc, A, Sy, Sz,Iyy,Izz,Iyz1,Iy,Iz,Iyz,I1,I2,Theta

################################
#CALCUL SUR TOUTES LES SECTIONS
################################

"""caracteritique_pale est un tabeau trois colonnes avec 
 1 -> coordinates
 2 -> cord
 3 -> twist angle """
def method1 (data_blade,section_cord_unitary):
    section_interpolated = interpolation_blade(section_cord_unitary)
    [Yc, Zc, A, Sy, Sz,Iyy,Izz,Iyz1,Iy,Iz,Iyz,I1,I2,Theta] = Calculus_geometrical (section_interpolated)
    y= data_blade[:, 0]
    c = data_blade[:, 1]
    alpha = data_blade[:, 2]    
    yc = np.empty(len(data_blade))
    zc = np.empty(len(data_blade))
    A = np.empty(len(data_blade))
    I1 = np.empty(len(data_blade))
    I2 = np.empty(len(data_blade))
    "Filling the tables"
    for i in range (len(y)):
        Iyyi = Iyy * c[i]**4
        Izzi = Izz * c[i]**4
        Iyzi = Iyz * c[i]**4
        Syi = Sy * c[i]**3
        Szi = Sz * c[i]**3
        Ai = A * c[i]*2
        yci = Szi/Ai
        zci = Syi/Ai
        yc.append(Szi/Ai)
        zc.append(Syi/Ai)
        A.append(Ai)
        Iyi = Iyyi - (zci**2)*A
        Izi = Izzi - A*yci**2
        Iyzi = Iyzi - Yc*Zc*A
        I1i = 1/2*(Izi+Iyi) - math.sqrt(((Iyi-Izi)/2)**2 + Iyzi**2)
        I2i = 1/2*(Izi+Iyi) + math.sqrt(((Iyi-Izi)/2)**2 + Iyzi**2)
        I1.append(I1i)
        I2.append(I2i)
    "Compiling tables"
    Table = np.stack((y,c,alpha,yc,zc,A,I1,I2))
    return Table

"""caracteritics_blade is a three-column table with 
 1 -> coordinates
 2 -> cord
 3 -> twist angle 
 4 -> Section points that will be interpolated"""
def method2 (data_blade):
    y= data_blade[:, 0]
    c = data_blade[:, 1]
    alpha = data_pale[:, 2]    
    yc = np.empty(len(data_blade))
    zc = np.empty(len(data_blade))
    A = np.empty(len(data_blade))
    I1 = np.empty(len(data_blade))
    I2 = np.empty(len(data_blade))
    "Filling the tables"
    for i in range (len(y)):
        [Yc, Zc, A, Sy, Sz,Iyy,Izz,Iyz1,Iy,Iz,Iyz,I1,I2,Theta] = Calculus_geometrical (interpolation_blade(data_blade[i][3]))
        Iyyi = Iyy * c[i]**4
        Izzi = Izz * c[i]**4
        Iyzi = Iyz * c[i]**4
        Syi = Sy * c[i]**3
        Szi = Sz * c[i]**3
        Ai = A * c[i]*2
        yci = Szi/Ai
        zci = Syi/Ai
        yc.append(yci)
        zc.append(zci)
        A.append(Ai)
        Iyi = Iyyi - (Zc**2)*A
        Izi = Izzi - A*Yc**2
        Iyzi = Iyzi - Yc*Zc*A
        I1i = 1/2*(Izi+Iyi) - math.sqrt(((Iyi-Izi)/2)**2 + Iyzi**2)
        I2i = 1/2*(Izi+Iyi) + math.sqrt(((Iyi-Izi)/2)**2 + Iyzi**2)
        I1.append(I1i)
        I2.append(I2i)
    "Compiling tables"
    Table = np.stack((y,c,alpha,yc,zc,A,I1,I2))
    return Table


Geometry calculation

In [ ]:
method = bool(input("Notice : \n method 1 : with constant section (True) \n method  2 : with variable section form (False) \n Careful to follow the methods notice by using one or the other \n chosen method : "))
if method == True :
    Table = method1(data_blade,section_cord_unitary)
else : 
    Table = method2(data_blade)

Stress Calculation

In [ ]:
tab = [[0,1],[1,2]]
Trial = method_trapeze(tab)
#Results
T = method_trapeze(L)
print(T)
N = method_trapeze(F)
M = method_trapeze(m)
T = method_trapeze()
#D = method_trapeze(method_trapeze(m/(E*I))) "I liste de I(x) et m liste de M(x)"


print("Shear force: ",T,"Normal force", N,"Moment" , M, "Deflexion", D)

#Root stress

D_root=30 #mm
Table1=method1(data_blade,section_cord_unitary)
I_root1=Table1[6][0]
I_root2=Table1[7][0]
M_root=M

#Stress formula at the root
sigma_root = M_root*D_root/2*I_root
